# 6. ML Models (XGBoost + LSTM templates)

XGBoost is implemented end-to-end.
LSTM section is a template placeholder.


In [ ]:

import pandas as pd
from pathlib import Path
from src.ml_models import train_xgb

DATA_DIR = Path("../data")
df = pd.read_csv(DATA_DIR/"nifty_features_5min.csv", parse_dates=["timestamp"])

df["y"] = (df["spot_ret"].shift(-1) > 0).astype(int)

feature_cols = [
    "ema_5","ema_15","avg_iv","iv_spread","pcr_oi","pcr_vol","futures_basis",
    "call_delta","call_gamma","call_vega","delta_neutral_ratio","gamma_exposure","regime"
]
for c in feature_cols:
    if c not in df.columns:
        df[c] = 0.0

df = df.dropna(subset=feature_cols+["y"]).reset_index(drop=True)

X = df[feature_cols].values
y = df["y"].values

split = int(len(df)*0.7)
xgb = train_xgb(X[:split], y[:split])

proba = xgb.predict_proba(X[split:])[:,1]
pred = (proba > 0.5).astype(int)
acc = (pred == y[split:]).mean()
print("XGBoost accuracy:", acc)


In [ ]:

import joblib
from pathlib import Path

MODEL_DIR = Path("../models")
MODEL_DIR.mkdir(exist_ok=True)
joblib.dump(xgb, MODEL_DIR/"xgboost_model.pkl")
print("Saved models/xgboost_model.pkl")


## LSTM Template

Build sequences of last 10 candles and train an LSTM:
- LSTM -> Dropout -> Dense(sigmoid)
- Filter trades when confidence > 0.5
